# Treinamento & Validação
Esse **Jupyter Notebook** tem como objetivo treinar modelos de *Machine Learning* com uma ou mais *features* e verificar quão bem esses modelos estão aprendendo com base na métrica de validação - [Erro Médio Absoluto](https://en.wikipedia.org/wiki/Mean_absolute_error).

---

# Classes "Training"
Como um dos requisitos da **GRIA** para o desafio era que os códigos fossem reaproveitados e documentados. Para satisfazer esses requisitos vamos criar a classe **Training** que vai ser Responsável pelo processo de treinar nossos dados em vários modelos diferentes e metrifica-los.

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import numpy as np


class Training:

  def split_data(self, x, y):
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=42)
    return x_train, x_valid, y_train, y_valid


  def predict_linear_regression(self, x_train, x_valid, y_train, y_valid):
    model = LinearRegression() # Instance.
    model.fit(x_train, y_train) # Training.
    salaries_predicted = model.predict(x_valid) # Predict salaries for x_valid.
    return salaries_predicted


  def predict_ridge_regression(self, x_train, x_valid, y_train, y_valid):
    model = Ridge(alpha=1.0) # Alpha = Learning Rate.
    model.fit(x_train, y_train) # Training.
    salaries_predicted = model.predict(x_valid)
    return salaries_predicted


  def predict_lasso_regression(self, x_train, x_valid, y_train, y_valid):
    model = Lasso(alpha=10, max_iter=1000, tol=0.1)
    model.fit(x_train, y_train) # Training.
    salaries_predicted = model.predict(x_valid)
    return salaries_predicted


  def predict_elasticnet_regression(self, x_train, x_valid, y_train, y_valid):
    model = ElasticNet(alpha=1, l1_ratio=0.5, tol=0.3)
    model.fit(x_train, y_train) # Training.
    salaries_predicted = model.predict(x_valid)
    return salaries_predicted


  def predict_random_forest_regressorn(self, x_train, x_valid, y_train, y_valid):
    model = RandomForestRegressor(n_jobs=-1) # Instance.
    model.fit(x_train, np.ravel(y_train)) # Training.
    salaries_predicted = model.predict(x_valid)
    return salaries_predicted


  def get_mae_scores(self, x, y):

    # KFold instance -  # shuffle=True, Shuffle (embaralhar) the data.
    kfold = KFold(
      n_splits=10,
      shuffle=True
    )

    # Models instances.
    randomForestRegressor = RandomForestRegressor(n_jobs=-1)
    linearRegression      = LinearRegression()
    elasticNet            = ElasticNet()
    ridge                 = Ridge()
    lasso                 = Lasso()

    # Apply cross-validation with KFold for all models.
    randomForestRegressor_result = abs(cross_val_score(randomForestRegressor, x, y, cv = kfold, scoring='neg_mean_absolute_error'))
    linearRegression_result      = abs(cross_val_score(linearRegression, x, y, cv = kfold, scoring='neg_mean_absolute_error'))
    elasticNet_result            = abs(cross_val_score(elasticNet, x, y, cv = kfold, scoring='neg_mean_absolute_error'))
    ridge_result                 = abs(cross_val_score(ridge, x, y, cv = kfold, scoring='neg_mean_absolute_error'))
    lasso_result                 = abs(cross_val_score(lasso, x, y, cv = kfold, scoring='neg_mean_absolute_error'))

    # Create a dictionary to store the Models.
    dic_models = {
      "randomForestRegressor": randomForestRegressor_result.mean(),
      "LinearRegression": linearRegression_result.mean(),
      "ElasticNet": elasticNet_result.mean(),
      "Ridge": ridge_result.mean(),
      "Lasso": lasso_result.mean()
    }
    bestModel = min(dic_models, key=dic_models.get) # Select the best model.

    print("MAE for Random Forest Regressor: {0}\nMAE for Linear Regression: {1}\nMAE for Ridge (L2) Regression: {2}\nMAE for Lasso (L1) Regression: {3}\nMAE for Elastic Net (L2 + L1) Regression: {4}".format(randomForestRegressor_result.mean(), linearRegression_result.mean(), elasticNet_result.mean(), ridge_result.mean(), lasso_result.mean()))
    print("The best model is: {0} with MAE value: {1}".format(bestModel, dic_models[bestModel]))


Agora vamos criar uma instância da classe **Training**:

In [26]:
# Training instance.
training = Training()

---

# 01 - Preparando os dados & Ambiente
Nessa etapa vamos preparar os dados e o ambiente (jupyter notebook) para **treinar** e **validar** nossos modelos.

---

## 01.1 - Baixando as Bibliotecas necessárias
Inicialmente vamos baixar as bibliotecas necessárias para nossa análise (Eu já tenho todas baixadas no meu ambiente virtual mas você pode remover o comentário e baixar para sua máquina local ou Ambiente Virtual).

In [3]:
#!pip install --upgrade -r ../requirements.txt --user

---

## 01.2 - Importando o módulo "Preprocessing"
Nós também vamos utilizar o módulo **"Preprocessing"** que foi criado na etapa de *Pré-Processamento*. Lembrem, que nós também temos que *Pré-Processar* os dados de *teste* como foi feito nos dados de treino e vamos utilizar os métodos de extrair e pegar os conjuntos de dados 

In [4]:
%run "../src/preprocessing.py"

In [5]:
preprocessing = Preprocessing()

---

## 01.3 - Extraindo os conjuntos de dados de treino e teste
Os conjuntos de dados de **treino** e **teste** vão ser utilizados quase que sempre para treinar e validar nossos modelos. Então, vamos deixar ambos já extraídos para trabalhos futuros.

In [6]:
# Extract training set.
preprocessing.extract_7z_data("../datasets/Train_rev1.7z")

File extracted!


In [7]:
# Extract testing set.
preprocessing.extract_7z_data("../datasets/Test_rev1.7z")

File extracted!


---

## 01.4 - Pegando o conjunto de dados de teste

Agora vamos pegar o conjunto de dados de teste disponibilizado pelo **Adzuna**.

**NOTE:**  
Vale lembrar que nesse conjunto de dados não tem as colunas (feature) **"SalaryRaw"** e **"SalaryNormalized"**.

In [8]:
df_testing = preprocessing.get_testing_data()

Testing data ready!


In [9]:
df_testing.info()
df_testing.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122463 entries, 0 to 122462
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  122463 non-null  int64 
 1   Title               122463 non-null  object
 2   FullDescription     122463 non-null  object
 3   LocationRaw         122463 non-null  object
 4   LocationNormalized  122463 non-null  object
 5   ContractType        33013 non-null   object
 6   ContractTime        90702 non-null   object
 7   Company             106202 non-null  object
 8   Category            122463 non-null  object
 9   SourceName          122463 non-null  object
dtypes: int64(1), object(9)
memory usage: 9.3+ MB


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,11888454,Business Development Manager,The Company: Our client is a national training...,"Tyne Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Teaching Jobs,cv-library.co.uk
1,11988350,Internal Account Manager,The Company: Founded in **** our client is a U...,"Tyne and Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Consultancy Jobs,cv-library.co.uk
2,12612558,Engineering Systems Analysts,Engineering Systems Analysts Surrey ****K Loca...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,12613014,CIS Systems Engineering Consultant,CIS Systems Engineering Consultant Bristol So...,"Bristol, South West, South West",Bristol,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
4,22454872,CNC Miller / Programmer Fanac,"CNC Miller / Programmer Fanac Fleet, Hampshire...","Fleet, Hampshire",Fleet,NaN,permanent,Gregory Martin International,Manufacturing Jobs,cv-library.co.uk


---

# 03 - Treinando & Validando os Loads
> Na parte de **Treinamento & Validação** nós vamos utilizar as colunas (features) já Pré-Processadas em cada **Load** para treinar vários modelos de Regressão e tentar encontrar o que nós dar o melhor resultado de acordo com os dados passados.

---

## 03.1 - Treinando & Validando o Load-v1
Bem, como nós sabemos no **load-v1** foi passado para a etapa de **treinamento & validação** a coluna (feature) **"Title"**. Ou seja, nós vamos ter as seguintes variáveis (features) para os nossos modelos:

 - **Variáveis Independente:**
   - Title *(com CountVectorizer):*
     - stop_words="english"
     - max_df=0.60 (Ignores terms that appear in MORE than 60% of documents)
     - min_df=0.05 (Ignores terms that appear in LESS than 5% of documents)
 - **Variáveis Dependente:**
   - SalaryNormalized (normalizada pelo a Adzuna)

**NOTE:**  
Esse vai ser o nosso **baseline model**.

**Pegando a variável dependente (target=y):**

In [10]:
df_training = preprocessing.get_training_data()

Training data ready!


In [11]:
y = df_training["SalaryNormalized"]

**Pegando a variável Independente (X):**

In [12]:
import scipy.sparse
df_title_vectorized = scipy.sparse.load_npz('../resources/processed_features/df_title_vectorized.npz')

In [13]:
x = df_title_vectorized

**Dividindo os dados em *dados de treino* e *dados de validação*:**

In [14]:
x_train, x_valid, y_train, y_valid = training.split_data(x, y)

**Testando a Métrica de validação MAE com Validação-Cruzada K-Fold**  
Por fim, vamos testar nossa **Métrica de Validação MAE (Mean Absolute Error)** para cada algoritmo, porém, utilizando uma **Validação-Cruzada** com **K-Fold**.

In [20]:
training.get_mae_scores(x_train, y_train)

MAE for Random Forest Regressor: 12072.775896817504
MAE for Linear Regression: 12218.397340449479
MAE for Ridge (L2) Regression: 13111.802628619404
MAE for Lasso (L1) Regression: 12218.662390720292
MAE for Elastic Net (L2 + L1) Regression: 12218.686131234477
The best model is: randomForestRegressor with MAE value: 12072.775896817504


**NOTE:**  
Bem, para esse conjunto de dados passado o modelo que mais se adequo aos dados foi o **Random Forest Regressor**.

---

# Resumos

 - **No *Load-v1* nós tinhamos as seguintes situações:**
   - Variáveis (features):
     - Independentes:
       - Title *(com CountVectorizer):*
         - stop_words="english"
         - max_df=0.60 (Ignores terms that appear in MORE than 60% of documents)
         - min_df=0.05 (Ignores terms that appear in LESS than 5% of documents)
     - Dependente:
       - SalaryNormalized (normalizada pelo a Adzuna)
   - *Como Métrica de Avaliação (MAE) tivemos os seguintes resultados (ordenados do menor para o maior):*
     -  MAE for RandomForestRegressor Model: 12037.451130584002
     -  MAE for Linear Regression Model: 12174.533594760238
     -  MAE for Ridge Model: 12174.538439350576
     -  MAE for Lasso Model: 12174.714368247884
     -  MAE for Elastic Net Model: 13057.741326652269

**Rodrigo Leite -** *drigols*